In [11]:
# imports
import numpy as np
import math
import operator
from random import randrange
import matplotlib.pyplot as plt

"""
#Euclidean Distance
def EuclideanDistance(instance1, instance2, length):
 distance = 0
 for i in range(length):
  distance += pow(instance2[i]-instance1[i],2)
 return math.sqrt(distance)

#Manhattan Distance
def ManhattanDistance(instance1, instance2, length):
    distance = 0
    for i in range(length):
        distance += abs(instance2[i]-instance1[i])
    return distance

#Minkowski distance with parameter p for power 
def MinkowskiDistance(instance1, instance2, length, p):
    distance = 0
    for i in range(length):
        distance += pow(abs(instance2[i]-instance1[i]), p)
    return pow(distance, 1/p)
"""

#Euclidean Distance
def EuclideanDistance(instance1, instance2):
    distance = 0
# for i in range(length):
    distance += pow(instance2-instance1,2)
    return math.sqrt(distance)

#Manhattan Distance
def ManhattanDistance(instance1, instance2):
    distance = 0
#    for i in range(length):
    distance += abs(instance2-instance1)
    return distance

#Minkowski distance with parameter p for power 
def MinkowskiDistance(instance1, instance2, p):
    distance = 0
#    for i in range(length):
    distance += pow(abs(instance2-instance1), p)
    return pow(distance, 1/p)


# Split a dataset into a train and test set
def train_test_split(dataset, split):
    train = list()
    train_size = split * len(dataset)
    dataset_copy = list(dataset)
    while len(train) < train_size:
        index = randrange(len(dataset_copy))
        train.append(dataset_copy.pop(index))
    return train, dataset_copy

# Split a dataset into $k$ folds
def cross_validation_split(dataset, folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / folds)
    for i in range(folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

#Get neighbors
def getNeighbors(trainingSet, testInstance, num_neighbors, distancetype, *args):
    distances = []
#    length = len(testInstance)-1
    if isinstance(testInstance, np.ndarray):
        length = testInstance.shape[0]
    else:
        length = 1
    print(length)
    for i in range(len(trainingSet)):
        if distancetype == "Euclidean":
            dist = EuclideanDistance(testInstance, trainingSet[i])
        elif distancetype == "Manhattan":
            dist = ManhattanDistance(testInstance, trainingSet[i])
        else:
            dist = MinkowskiDistance(testInstance, trainingSet[i], *args)
        distances.append((trainingSet[i],dist))
    distances.sort(key=operator.itemgetter(1))
 #return distances
    neighbors = []
    for x in range(num_neighbors):
        neighbors.append(distances[x][0])
    return neighbors

#Regression by taking mean from neighbors (Regression problem)
def getRegression(neighbors):
#    print(f'N = {neighbors}')
#    output_values = [row[-1] for row in neighbors]
#    print(f'O = {output_values}')
#    return sum(output_values) / float(len(output_values))
    return sum(neighbors) / float(len(neighbors))

###### Accuracy methods for Regression problems ########

# Calculate mean absolute error (MAE) #
def mae_metric(actual, predicted):
 sum_error = 0.0
 for i in range(len(actual)):
  sum_error += abs(predicted[i] - actual[i])
 return sum_error / float(len(actual))

# Calculate root mean squared error #
def rmse_metric(actual, predicted):
 sum_error = 0.0
 for i in range(len(actual)):
  prediction_error = predicted[i] - actual[i]
  sum_error += (prediction_error ** 2)
 mean_error = sum_error / float(len(actual))
 return math.sqrt(mean_error)

# Load dataset
dataset = np.load('./data2.npz')
data1_x = np.array(dataset['X'])
data1_y = np.array(dataset['y'])
    
def main():
 # Splitting dataset between Training and Testing Set
    split = 0.5
    trainingSet, testSet = train_test_split(data1_x, split)
    
#    print(f'trainingSet={trainingSet}')
#    print(f'testSet={testSet}')
 
    #generate predictions
    predictions = []
    num_neighbors = 5
    print(len(trainingSet), testSet[0])
    for i in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[i], num_neighbors, "Euclidean")
#         output = getRegression(neighbors)
#         predictions.append(output)
# #        print('> predicted = %.3f, actual = %.3f' % (output, testSet[i]))

#     print('')
#     MAE = mae_metric(testSet, predictions)
#     print('MAE: %.5f' % MAE)
#     RMSE = rmse_metric(testSet, predictions)
#     print('RMSE: %.5f' % RMSE)
    
# #    print(testSet.shape)
# #    print(len(predictions))
    
#     plt.scatter(data1_x, data1_y, s = 5, label = 'True')
#     plt.scatter(testSet, predictions, color="red", s = 5, label='Predicted')
#     plt.xlabel('X-axis')
#     plt.ylabel('Y-axis')
#     plt.legend()
#     plt.show

main()

500 [ 1.75801899 -0.57290817]
2


TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
def EuclideanDistance(instance1, instance2, length):
	distance = 0
	for i in range(length):
		distance += pow(instance2[i] - instance1[i],2)
	return math.sqrt(distance)
    
def getNeighbors(trainingSet, testInstance, num_neighbors, distancetype, *args):
    distances = []
    length = len(trainingSet[0])
    testInstance = list(testInstance)
    # testInstance = list(np.squeeze(testInstance))
    # length = testInstance
    for i in range(len(trainingSet)):
        if distancetype == "Euclidean":
            dist = ntrainingSet[i]
            dist = EuclideanDistance(testInstance, trainingSet[i], length)
        elif distancetype == "Manhattan":
            dist = ManhattanDistance(testInstance, trainingSet[i], length)
        else:
            dist = MinkowskiDistance(testInstance, trainingSet[i], length, *args)
        distances.append((trainingSet[i],dist))
    distances.sort(key=operator.itemgetter(1))
	#return distances
    neighbors = []
    for x in range(num_neighbors):
        neighbors.append(distances[x][0])
    return neighbors

In [15]:
t = np.array([[1,2]])
print(t.shape)

(1, 1)


[1]